# $l_1$--regularization
$\newcommand{\n}[1]{\left\|#1 \right\|}$ 
$\renewcommand{\a}{\alpha}             $ 
$\renewcommand{\b}{\beta}              $ 
$\renewcommand{\c}{\gamma}             $ 
$\renewcommand{\d}{\delta}             $ 
$\newcommand{\D}{\Delta}               $ 
$\newcommand{\la}{\lambda}             $ 
$\renewcommand{\t}{\tau}               $ 
$\newcommand{\s}{\sigma}               $ 
$\newcommand{\e}{\varepsilon}          $ 
$\renewcommand{\th}{\theta}            $ 
$\newcommand{\x}{\bar x}               $ 
$\newcommand{\R}{\mathbb R}            $ 
$\newcommand{\N}{\mathbb N}            $ 
$\newcommand{\Z}{\mathbb Z}            $ 
$\newcommand{\E}{\mathcal E}           $ 
$\newcommand{\lr}[1]{\left\langle #1\right\rangle}$
$\newcommand{\nf}[1]{\nabla f(#1)}     $
$\newcommand{\hx}{\hat x}               $
$\newcommand{\hy}{\hat y}               $
$\DeclareMathOperator{\prox}{prox}      $
$\DeclareMathOperator{\argmin}{argmin}  $
$\DeclareMathOperator{\dom}{dom}        $
$\DeclareMathOperator{\id}{Id}          $
$\DeclareMathOperator{\conv}{conv}      $

We want to minimize:
$$
\min_x \frac 1 2 ||Ax-b||^2 + \lambda  ||x||_1 =: f(Ax)+g(x),
$$
where $A\in \R^{m\times n}$ is a matrix data, $b\in \R^m$ is a given observation, and $x\in \R^n$ is an unknown signal.

We can rewrite the problem above in a primal-dual form as follows:
$$
\min_x \max_y g(x)+(Ax,y)-f^*(y),
$$
where $f(x) = \frac 1 2 ||x-b||^2$, $f^*(y) = \frac 1 2 ||y||^2 + (b,y) = \frac 1 2 ||y+b||^2 -\frac{1}{2}||b||^2$ and $g(x) = \lambda ||x||_1$.

For the problem in a primal-dual form we apply primal-dual methods and for the problem in a primal form we apply proximal gradient method and FISTA. For this we set $h(x) = f(Ax)$ and get $\nabla h(x) = A^*(Ax-b)$.

In [26]:
import matplotlib.pyplot as plt

from opt_operators import *
from algorithms import *
from pd_algorithms import *

%reload_ext autoreload
%autoreload 2

We construct our data in one of the following ways below. We will use a fixed random generator for all our data to make experiments reproducible.

In [2]:
gen = 100

In [3]:
n = 1000
m = 200
s = 10
la = 0.1

np.random.seed(gen)
A = np.random.normal(0,1, (m,n))
np.random.seed(gen)
w = np.random.uniform(-10,10, n)
w[s:] = 0
np.random.seed(gen)
w = np.random.permutation(w)

In [3]:
n = 2000
m = 1000
s = 100
la = 0.1

np.random.seed(gen)
A = np.random.normal(0,1, (m,n))

np.random.seed(gen)
w = np.random.normal(0,1, n)
w[s:] = 0
np.random.seed(gen)
w = np.random.permutation(w)

For the problem below we took $p = 0.5$ or $p = 0.9$

In [32]:
n = 5000
m = 1000
s = 50
la = 0.1

np.random.seed(gen)
B = np.random.normal(0,1, (m,n))
p = 0.5
A = np.zeros((m,n))
A[:,0] = B[:,0]/np.sqrt(1-p**2)
for j in np.arange(1,n):
    A[:,j] = p*A[:,j-1] + B[:,j]

np.random.seed(gen)
w = np.random.uniform(-10,10, n)
w[s:] = 0
np.random.seed(gen)
w = np.random.permutation(w)

Define $\nu$ and $b$

In [13]:
np.random.seed(gen)
nu = np.random.normal(0,0.1, m)
b = A.dot(w) + nu

Define all ingredients for primal-dual methods and proximal gradient-like

In [14]:
def f_conj(y):
    return 0.5*(y+b).dot(y+b)

def g(x):
    return la*LA.norm(x,1)

def prox_g(x, rho):
    return prox_norm_1(x,la*rho)

def prox_f_conj(y, rho):
    return (y - rho*b)/(1+rho)

# define energy 
def J(x,y):
    t = A.dot(x)-b
    return 0.5* t.dot(t) + la* LA.norm(x,1)



#### for proximal gradient method and FISTA
def dh(x):
    return A.T.dot(A.dot(x)-b)

def F(x):
    return J(x,1)

Compute matrix norm of operator $A$. This may be long if you consider large-scale problems.

In [15]:
L = np.sqrt(np.max(LA.eigh(A.dot(A.T))[0]))

In [22]:
# number of iterations
N = 10000

# starting points
x0 = np.zeros(n)
y0 = -b

# step size for PDA
tau = 1./L
sigma = 1./L

# step size for PDAL (doesn't require any expensive computation)
tau0 = np.sqrt(m)/LA.norm(A)

# step size for PGM and FISTA
alpha = 1./L**2

In [ ]:
ans1 = pd(J, prox_g, prox_f_conj, A, x0, y0, 0.05*sigma, 20*tau, numb_iter=N)
ans2 =  pd_linesearch_dual_is_square_norm(J, prox_g, -b, A, x0, y0 , tau, 1./400, numb_iter=N)
ans3 = fista(F, dh, prox_g, x0, alpha, numb_iter=N)
ans4 = prox_grad(F, dh, prox_g, x0, alpha, numb_iter=N)

To see the plots of residual $f(Ax)+g(x) - f(Ax^*)-g(x^*)$. For simplicity, we just set $f(Ax^*)+g(x^*)$ as the smallest number among all energy values for all methods during all iterations. Alternatively, you can increase number of iterations to obtain even better ground truth solution.

In [24]:
t = min(ans1[0]+ans2[0]+ans3[0]+ans4[0])

plt.plot(ans1[0]-t, 'b',)
plt.plot(ans2[0]-t, 'r',)
plt.plot(ans3[0]-t, 'c',)
plt.plot(ans4[0]-t, 'g',)
plt.yscale('log')
plt.show()

Check how sparse is your solution and how it is different from $w$:

In [25]:
plt.plot(w, 'g')
plt.plot(ans2[1], 'k')
plt.show()

Nice plots as in the paper:

In [11]:
import matplotlib as mpl
mpl.rc('lines', linewidth=2)
mpl.rcParams.update(
    {'font.size': 13, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
mpl.rcParams['xtick.major.pad'] = 2
mpl.rcParams['ytick.major.pad'] = 2

t = min(ans1[0]+ans2[0]+ans3[0]+ans4[0])
#r = 10000
plt.plot(ans1[0]-t, 'b',label = 'PDA')
plt.plot(ans2[0]-t, 'r', label = 'PDAL')
plt.plot(ans3[0]-t, 'c',label = 'FISTA')
plt.plot(ans4[0]-t, 'g',label = 'PGM')
plt.yscale('log')
#plt.xscale('log')
plt.xlabel(u' iterations ')
plt.ylabel('$\phi(x)-\phi^*$')

plt.legend()
plt.savefig('figures/lasso-2.pdf')
plt.show()